# Run this cell to regenerate the v2 files that are used in the rsvp.py script

In [ ]:
# Just set these parameters:
# Normally, for the LPP read script, we use:
# black_screen = 0.05
# word_duration = 0.25
# end_of_sentence_delay = 0.5
# word_bs = black_screen + word_duration
# end_of_chapter_duration = 2

# Here, I setup a bit more word duration

In [ ]:
import pandas as pd
import numpy as np

black_screen = 0.10
word_duration = 0.5
end_of_sentence_delay = 0.8
word_bs = black_screen + word_duration
end_of_chapter_duration = 2

dict_end_chapter = {
    1:[433,1087],
    2:[737,1400],
    3:[710,1345],
    4:[1090,1357],
    5:[753],
    6:[716,951,1271,1555],
    7:[200,1278],
    8:[95,703],
    9:[1357],
}

for i in np.arange(1,10):
    with open(f'./text_lpp/new_test_run{i}.txt') as temp_file:

        lpp = temp_file.read().splitlines() 


    df = pd.DataFrame(lpp)
    next_cap = False

    for index, row in df.iterrows():
        # First word
        if index == 0:
            df.at[index,0] = str(row.str.capitalize()[0])
        if next_cap == True:
            df.at[index,0] = str(row.str.capitalize()[0])
        if str(row).__contains__('.') or str(row).__contains__('?') or str(row).__contains__('!'):
            next_cap = True
        else:
            next_cap = False

    df.columns = ['word']
    end = (df.shape[0] * word_bs) + 0.7
    df['onset'] = np.arange(0.7, end, word_bs)
    df['duration'] = np.ones(df.shape[0]) * word_duration
    
    
    
    df.to_csv(f'./txt_clean/run{i}_clean.tsv', sep='\t', index=False)
    
    
    # Create a dataframe where the duration of the black screen after the end of the sentence is longer.

    df_sentence_end = pd.DataFrame(columns = df.columns, data = copy.deepcopy(df.values))
    for index, row in df.iterrows():
        if str(row.word).__contains__('.') or str(row.word).__contains__('?') or str(row.word).__contains__('!'):
            # df_sentence_end.at[index, 'onset'] = row.onset + end_of_sentence_delay # Add the delay from this line
            # And for every next onset
            for j in np.arange(index+1, df.shape[0]):
                df_sentence_end.at[j, 'onset'] = df_sentence_end.at[j, 'onset'] + end_of_sentence_delay
                ww = df_sentence_end.at[j, 'word']
        if index+2 in dict_end_chapter[i]:
            print(f'Adding 2s after the word {row.word} \n')
            for j in np.arange(index+1, df.shape[0]):
                df_sentence_end.at[j, 'onset'] = df_sentence_end.at[j, 'onset'] + end_of_chapter_duration
                ww = df_sentence_end.at[j, 'word']

    
    df_sentence_end.to_csv(f'./v2/run{i}_v2_0.25_0.5.tsv',sep='\t',index=False)
    
